## Section 1: Setup & Load Graph

Load libraries and reconstruct the graph from Notebook 01

In [ ]:
import os
import sys
import asyncio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from typing import Dict, List, Tuple
from collections import defaultdict

import httpx
import redis.asyncio as redis
from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable

# Configure visualization
plt.style.use('dark_background')
%matplotlib inline

print("✅ Libraries loaded")

In [ ]:
# Load environment and initialize connections (same as Notebook 01)
REDIS_URL = os.getenv('REDIS_URL', 'redis://redis:6379')
MEMGRAPH_URL = os.getenv('MEMGRAPH_URL', 'bolt://memgraph:7687')
BACKEND_API_URL = os.getenv('BACKEND_API_URL', 'http://code-graph-http:8000')

class GraphConnections:
    """Manages connections to Redis, Memgraph, and Backend API"""
    def __init__(self, redis_url: str, memgraph_url: str, api_url: str):
        self.redis_url = redis_url
        self.memgraph_url = memgraph_url
        self.api_url = api_url
        self.redis_client = None
        self.memgraph_driver = None
        self.http_client = None
    
    async def connect(self):
        try:
            self.redis_client = await redis.from_url(self.redis_url, decode_responses=True)
            await self.redis_client.ping()
            print("✅ Redis connected")
        except Exception as e:
            print(f"⚠️  Redis connection failed: {e}")
        
        try:
            self.memgraph_driver = GraphDatabase.driver(self.memgraph_url)
            with self.memgraph_driver.session() as session:
                session.run("RETURN 1").consume()
            print("✅ Memgraph connected")
        except ServiceUnavailable:
            print(f"⚠️  Memgraph not available")
            self.memgraph_driver = None
        except Exception as e:
            print(f"⚠️  Memgraph connection failed: {e}")
            self.memgraph_driver = None
        
        self.http_client = httpx.AsyncClient(base_url=self.api_url)
        print("✅ HTTP client ready")

connections = GraphConnections(REDIS_URL, MEMGRAPH_URL, BACKEND_API_URL)
loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)
loop.run_until_complete(connections.connect())

print("\n✅ All systems initialized!")

In [ ]:
# Load graph data
async def get_all_nodes():
    try:
        response = await connections.http_client.get('/api/graph/nodes/search?limit=1000')
        data = response.json()
        return pd.DataFrame(data.get('results', []))
    except Exception as e:
        print(f"Error fetching nodes: {e}")
        return pd.DataFrame()

async def get_all_relationships():
    try:
        response = await connections.http_client.get('/api/graph/relationships?limit=5000')
        data = response.json()
        return data.get('results', [])
    except Exception as e:
        print(f"Error fetching relationships: {e}")
        return []

# Fetch data
nodes_df = loop.run_until_complete(get_all_nodes())
relationships = loop.run_until_complete(get_all_relationships())

# Build NetworkX graph
G = nx.DiGraph()

if not nodes_df.empty:
    for idx, row in nodes_df.iterrows():
        G.add_node(row['name'], 
                   type=row.get('type'), 
                   file=row.get('file'),
                   language=row.get('language'))
    print(f"✅ Added {len(nodes_df)} nodes")

for rel in relationships:
    if rel.get('source_name') in G and rel.get('target_name') in G:
        G.add_edge(rel['source_name'], rel['target_name'], 
                   relationship_type=rel.get('relationship_type'))

print(f"✅ Added {G.number_of_edges()} edges")
print(f"\nGraph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

## Section 2: Degree Centrality

Degree centrality measures how directly connected a node is to other nodes.

In [ ]:
# Calculate degree centrality
degree_centrality = nx.degree_centrality(G)

# Convert to DataFrame for analysis
degree_df = pd.DataFrame.from_dict(degree_centrality, orient='index', columns=['degree_centrality'])
degree_df = degree_df.sort_values('degree_centrality', ascending=False)

print("🎯 Top 15 Nodes by Degree Centrality\n")
print(degree_df.head(15).to_string())

print(f"\n📊 Statistics:")
print(f"   Mean: {degree_df['degree_centrality'].mean():.4f}")
print(f"   Median: {degree_df['degree_centrality'].median():.4f}")
print(f"   Std Dev: {degree_df['degree_centrality'].std():.4f}")
print(f"   Max: {degree_df['degree_centrality'].max():.4f}")

In [ ]:
# Visualize degree centrality distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(degree_df['degree_centrality'], bins=50, color='#4F46E5', alpha=0.7)
axes[0].set_xlabel('Degree Centrality', fontsize=11)
axes[0].set_ylabel('Number of Nodes', fontsize=11)
axes[0].set_title('Degree Centrality Distribution', fontsize=12, fontweight='bold')
axes[0].set_yscale('log')

# Top 20 nodes
top_20_degree = degree_df.head(20)
axes[1].barh(range(len(top_20_degree)), top_20_degree['degree_centrality'], color='#4F46E5')
axes[1].set_yticks(range(len(top_20_degree)))
axes[1].set_yticklabels(top_20_degree.index, fontsize=9)
axes[1].set_xlabel('Degree Centrality', fontsize=11)
axes[1].set_title('Top 20 Nodes by Degree Centrality', fontsize=12, fontweight='bold')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

## Section 3: Betweenness Centrality

Identifies nodes that act as bridges between different parts of the graph (bottlenecks).

In [ ]:
# Calculate betweenness centrality
print("⏳ Computing betweenness centrality (may take a moment)...")
betweenness_centrality = nx.betweenness_centrality(G)

betweenness_df = pd.DataFrame.from_dict(betweenness_centrality, orient='index', columns=['betweenness_centrality'])
betweenness_df = betweenness_df.sort_values('betweenness_centrality', ascending=False)

print("\n🎯 Top 15 Nodes by Betweenness Centrality (Bottlenecks)\n")
print(betweenness_df.head(15).to_string())

print(f"\n📊 Statistics:")
print(f"   Mean: {betweenness_df['betweenness_centrality'].mean():.4f}")
print(f"   Median: {betweenness_df['betweenness_centrality'].median():.4f}")
print(f"   Max: {betweenness_df['betweenness_centrality'].max():.4f}")

print("\n💡 Insight: These functions are critical bridges in the call graph.")
print("   Modifying them affects many other functions.")

In [ ]:
# Visualize betweenness centrality
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(betweenness_df['betweenness_centrality'], bins=50, color='#EC4899', alpha=0.7)
axes[0].set_xlabel('Betweenness Centrality', fontsize=11)
axes[0].set_ylabel('Number of Nodes', fontsize=11)
axes[0].set_title('Betweenness Centrality Distribution', fontsize=12, fontweight='bold')
axes[0].set_yscale('log')

# Top 20 nodes
top_20_betweenness = betweenness_df[betweenness_df['betweenness_centrality'] > 0].head(20)
axes[1].barh(range(len(top_20_betweenness)), top_20_betweenness['betweenness_centrality'], color='#EC4899')
axes[1].set_yticks(range(len(top_20_betweenness)))
axes[1].set_yticklabels(top_20_betweenness.index, fontsize=9)
axes[1].set_xlabel('Betweenness Centrality', fontsize=11)
axes[1].set_title('Top Bottleneck Functions', fontsize=12, fontweight='bold')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

## Section 4: Closeness Centrality

Measures average distance to all other nodes (how central a node is in terms of reachability).

In [ ]:
# For directed graphs, use weakly connected components
print("⏳ Computing closeness centrality...")
closeness_centrality = nx.closeness_centrality(G)

closeness_df = pd.DataFrame.from_dict(closeness_centrality, orient='index', columns=['closeness_centrality'])
closeness_df = closeness_df.sort_values('closeness_centrality', ascending=False)

print("\n🎯 Top 15 Nodes by Closeness Centrality\n")
print(closeness_df.head(15).to_string())

print(f"\n📊 Statistics:")
print(f"   Mean: {closeness_df['closeness_centrality'].mean():.4f}")
print(f"   Median: {closeness_df['closeness_centrality'].median():.4f}")
print(f"   Max: {closeness_df['closeness_centrality'].max():.4f}")

print("\n💡 Insight: Functions close to all others are good candidates for refactoring.")

## Section 5: PageRank

Identifies important nodes based on graph structure and incoming edges (like Google's PageRank algorithm).

In [ ]:
# Calculate PageRank
print("⏳ Computing PageRank...")
pagerank = nx.pagerank(G)

pagerank_df = pd.DataFrame.from_dict(pagerank, orient='index', columns=['pagerank'])
pagerank_df = pagerank_df.sort_values('pagerank', ascending=False)

print("\n🎯 Top 15 Nodes by PageRank\n")
print(pagerank_df.head(15).to_string())

print(f"\n📊 Statistics:")
print(f"   Mean: {pagerank_df['pagerank'].mean():.6f}")
print(f"   Median: {pagerank_df['pagerank'].median():.6f}")
print(f"   Max: {pagerank_df['pagerank'].max():.6f}")

print("\n💡 Insight: High PageRank nodes are called by many other important nodes.")

In [ ]:
# Visualize PageRank
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(pagerank_df['pagerank'], bins=50, color='#8B5CF6', alpha=0.7)
axes[0].set_xlabel('PageRank Score', fontsize=11)
axes[0].set_ylabel('Number of Nodes', fontsize=11)
axes[0].set_title('PageRank Distribution', fontsize=12, fontweight='bold')
axes[0].set_yscale('log')

# Top 20 nodes
top_20_pagerank = pagerank_df.head(20)
axes[1].barh(range(len(top_20_pagerank)), top_20_pagerank['pagerank'], color='#8B5CF6')
axes[1].set_yticks(range(len(top_20_pagerank)))
axes[1].set_yticklabels(top_20_pagerank.index, fontsize=9)
axes[1].set_xlabel('PageRank Score', fontsize=11)
axes[1].set_title('Top 20 Nodes by PageRank', fontsize=12, fontweight='bold')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

## Section 6: Compare All Centrality Measures

Create a comprehensive comparison of all centrality metrics for the top nodes.

In [ ]:
# Combine all metrics
centrality_combined = pd.DataFrame({
    'degree': degree_df['degree_centrality'],
    'betweenness': betweenness_df['betweenness_centrality'],
    'closeness': closeness_df['closeness_centrality'],
    'pagerank': pagerank_df['pagerank']
})

# Normalize for better comparison
centrality_normalized = centrality_combined.copy()
for col in centrality_normalized.columns:
    centrality_normalized[col] = (centrality_normalized[col] - centrality_normalized[col].min()) / (centrality_normalized[col].max() - centrality_normalized[col].min())

# Calculate average centrality
centrality_normalized['avg_centrality'] = centrality_normalized.mean(axis=1)
centrality_normalized = centrality_normalized.sort_values('avg_centrality', ascending=False)

print("🎯 Top 20 Nodes - Normalized Centrality Scores\n")
print(centrality_normalized.head(20).to_string())

In [ ]:
# Analyze correlations between metrics
correlation_matrix = centrality_combined.corr()

print("\n📊 Correlation Matrix Between Centrality Measures\n")
print(correlation_matrix.to_string())

# Visualize correlation
plt.figure(figsize=(8, 6))
import seaborn as sns
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
            square=True, cbar_kws={'label': 'Correlation'})
plt.title('Centrality Metrics Correlation', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 Insight: High correlations mean metrics agree on importance.")
print("   Low correlations reveal different aspects of centrality.")

## Section 7: Visualize Critical Nodes

Create graph visualizations highlighting high-centrality nodes.

In [ ]:
# Get top 10 nodes by average centrality
top_critical_nodes = centrality_normalized.head(10).index.tolist()

# Create subgraph with critical nodes and their neighbors
neighbors = set()
for node in top_critical_nodes:
    neighbors.update(G.predecessors(node))
    neighbors.update(G.successors(node))

subgraph_nodes = top_critical_nodes + list(neighbors)
G_sub = G.subgraph(subgraph_nodes).copy()

print(f"📊 Critical nodes subgraph:")
print(f"   Nodes: {G_sub.number_of_nodes()}")
print(f"   Edges: {G_sub.number_of_edges()}")

# Create visualization
plt.figure(figsize=(16, 12))

# Layout
pos = nx.spring_layout(G_sub, k=2, iterations=50, seed=42)

# Node colors based on PageRank
node_colors = [pagerank.get(node, 0) for node in G_sub.nodes()]

# Node sizes based on degree centrality
node_sizes = [degree_centrality.get(node, 0.1) * 3000 for node in G_sub.nodes()]

# Draw network
nx.draw_networkx_edges(G_sub, pos, edge_color='gray', alpha=0.3, arrows=True, 
                        arrowsize=15, arrowstyle='->', width=1.5)

nodes = nx.draw_networkx_nodes(G_sub, pos, node_color=node_colors, node_size=node_sizes,
                               cmap='YlOrRd', alpha=0.9)

# Labels for critical nodes only
labels = {node: node if node in top_critical_nodes else '' for node in G_sub.nodes()}
nx.draw_networkx_labels(G_sub, pos, labels, font_size=9, font_weight='bold')

plt.colorbar(nodes, label='PageRank', shrink=0.8)
plt.title('Critical Functions and Dependencies', fontsize=16, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

print("\n✅ Critical node visualization complete")

## Summary

This notebook demonstrated centrality analysis for identifying critical functions:

1. ✅ **Degree Centrality** - Direct connectivity (hub functions)
2. ✅ **Betweenness Centrality** - Bottleneck functions (bridges)
3. ✅ **Closeness Centrality** - Central functions (average distance)
4. ✅ **PageRank** - Importance based on incoming edges
5. ✅ **Comparison** - Correlation analysis of all metrics
6. ✅ **Visualization** - Critical node subgraph and dependencies

### Key Takeaways:
- **High degree + high betweenness**: Core library functions (high refactoring risk)
- **High PageRank only**: Important but not central (good refactoring candidates)
- **High closeness**: Hub functions (consider dependency injection)

### Next Step:
- **Notebook 03**: Community Detection (identify module boundaries)